In [1]:
import astropy.units as u
from astroduet.duet_telescope import load_telescope_parameters
from astroduet.duet_sensitivity import src_rate, bgd_sky_qe_rate, bgd_electronics, calc_exposure
from astroduet.bbmag import bb_abmag_fluence
import numpy as np
from matplotlib import pyplot as plt
from astroduet.bbmag import bb_abmag_fluence, bb_abmag

import astroduet.config as config
from astroduet.background import background_pixel_rate




from astropy.visualization import quantity_support
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

In [2]:
duet = config.Telescope()
duet.info()
print()
neff = duet.neff
print('Number of effective bgd pixels: {}'.format(neff))
duet.read_noise = 3

-----
DUET Telescope State:
Physical Entrance Pupil: 26.0 cm
Effective EPD: 24.2 cm
Effective Area: 459.9605804120816 cm2
Transmission Efficiency: 0.8166518036622619

Pixel size: 6.4 arcsec
PSF FWHM: 5.0 arcsec
Pointing jitter: 5.0 arcsec
Effective PSF FWHM: 7.0710678118654755 arcsec
N_eff: 3.3581908798935647

Band 1: {'eff_wave': <Quantity 202.56878682 nm>, 'eff_width': <Quantity 53.32814342 nm>}
Bandpass 1: [175.90471511 229.23285853] nm
Band 2: {'eff_wave': <Quantity 281.7531854 nm>, 'eff_width': <Quantity 68.16239088 nm>}
Bandpass 2: [247.67198996 315.83438085] nm


Dark current: 0.0115 ph / s
Read noise (RMS per read): 7
-----

Number of effective bgd pixels: 3.3581908798935647


In [16]:
# Band one
bandone = duet.bandpass1

print('For 15e3 K source')
for swiftmag in np.arange(19, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
    band1_rate = duet.fluence_to_rate(band1_fluence)

    if (band1_rate.value < 1.0):
        bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone)
        print('Band 1 Photometric Zero Point (Swift UVW2): {}'.format(swiftmag))
        print('Band 1 Photometric Zero Point: {}'.format(bbmag1))
        break
print()


For 15e3 K source
Band 1 Photometric Zero Point (Swift UVW2): 20.800000000000026
Band 1 Photometric Zero Point: 20.800470091092407 mag(AB)



In [17]:
# Band Two
bandtwo = duet.bandpass2

print('For 15e3 K source')
for swiftmag in np.arange(19, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
    band2_rate = duet.fluence_to_rate(band2_fluence)
    if (band2_rate.value < 1.0):
        foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandtwo = bandtwo)
        print('Band 2 Photometric Zero Point: {}'.format(swiftmag))
        print('Band 2 Photometric Zero Point: {}'.format(bbmag2))
        break
print()       


Band 2 Photometric Zero Point: 21.00000000000003
Band 2 Photometric Zero Point: 20.588257363507964 mag(AB)



In [58]:
# Individual channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.
# Assume read noise is 3 e-
duet = config.Telescope()
duet.read_noise=3

[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
print('Sky background rate per pixel band1: {}'.format(bgd_band1))
print('Sky background rate per pixel band2: {}'.format(bgd_band2))

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for nframes in np.arange(1, 5):
    snr = 100
    swiftmag = 20 
    while snr > siglimit:
        swiftmag += dmag
        band1_fluence, foo = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
        band1_rate = duet.fluence_to_rate(band1_fluence)
        snr = duet.calc_snr(exposure, band1_rate, bgd_band1, nint=nframes)
    bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone)
    print('Band 1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
print()

for nframes in np.arange(1, 5):
    snr = 100
    swiftmag = 20 
    while snr > siglimit:
        swiftmag += dmag
        foo, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
        band2_rate = duet.fluence_to_rate(band2_fluence)
        snr = duet.calc_snr(exposure, band2_rate, bgd_band2, nint=nframes)
    foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandtwo = bandtwo)
    print('Band 2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))
print()


Sky background rate per pixel band1: 0.030369732491096913 ph / s
Sky background rate per pixel band2: 0.2478588509265617 ph / s


Band 1 300.0 s 5-σ magnitude limit: 22.650470091092423 mag(AB)
Band 1 600.0 s 5-σ magnitude limit: 23.15047009109243 mag(AB)
Band 1 900.0 s 5-σ magnitude limit: 23.40047009109243 mag(AB)
Band 1 1200.0 s 5-σ magnitude limit: 23.600470091092433 mag(AB)

Band 2 300.0 s 5-σ magnitude limit: 21.83825736350797 mag(AB)
Band 2 600.0 s 5-σ magnitude limit: 22.238257363507977 mag(AB)
Band 2 900.0 s 5-σ magnitude limit: 22.488257363507977 mag(AB)
Band 2 1200.0 s 5-σ magnitude limit: 22.68825736350798 mag(AB)


Band 1 300-s 5-σ magnitude limit: 22.65047009109241 mag(AB)
Band 1 600-s 5-σ magnitude limit: 23.150470091092416 mag(AB)
Band 1 900-s 5-σ magnitude limit: 23.400470091092416 mag(AB)
Band 1 1200-s 5-σ magnitude limit: 23.60047009109242 mag(AB)

Band 2 300-s 5-σ magnitude limit: 21.838257363507957 mag(AB)
Band 2 600-s 5-σ magnitude limit: 22.238257363507962 mag(AB)

In [60]:
# Individual channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.
duet = config.Telescope()

# Account for the fact that you're co-adding the two frames here:
duet.read_noise = (2**0.5) * 3
duet.info()
[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
print('Sky background rate per pixel band1: {}'.format(bgd_band1))
print('Sky background rate per pixel band2: {}'.format(bgd_band2))

tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for nframes in np.arange(1, 5):
    snr = 100
    swiftmag = 20 
    while snr > siglimit:
        swiftmag += dmag
        band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
        band1_rate = duet.fluence_to_rate(band1_fluence)
        band2_rate = duet.fluence_to_rate(band2_fluence)
        src_rate = band1_rate + band2_rate
        
        snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
    bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)
    print('Stacked Bands {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
print()




-----
DUET Telescope State:
Physical Entrance Pupil: 26.0 cm
Effective EPD: 24.2 cm
Effective Area: 459.9605804120816 cm2
Transmission Efficiency: 0.8166518036622619

Pixel size: 6.4 arcsec
PSF FWHM: 5.0 arcsec
Pointing jitter: 5.0 arcsec
Effective PSF FWHM: 7.0710678118654755 arcsec
N_eff: 3.3581908798935647

Band 1: {'eff_wave': <Quantity 202.56878682 nm>, 'eff_width': <Quantity 53.32814342 nm>}
Bandpass 1: [175.90471511 229.23285853] nm
Band 2: {'eff_wave': <Quantity 281.7531854 nm>, 'eff_width': <Quantity 68.16239088 nm>}
Bandpass 2: [247.67198996 315.83438085] nm


Dark current: 0.0115 ph / s
Read noise (RMS per read): 4.242640687119286
-----
Sky background rate per pixel band1: 0.030369732491096913 ph / s
Sky background rate per pixel band2: 0.2478588509265617 ph / s


Stacked Bands 300.0 s 5-σ magnitude limit: 22.80047009109242 mag(AB)
Stacked Bands 600.0 s 5-σ magnitude limit: 23.200470091092427 mag(AB)
Stacked Bands 900.0 s 5-σ magnitude limit: 23.450470091092434 mag(AB)
Stack

In [12]:
# FOM: Time to 22 ABmag for this configuration for DUET1
# Old FOM for this 
duet = config.Telescope()
duet.read_noise = 3

[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
bandone = duet.bandpass1
bandtwo = duet.bandpass2

siglimit=5
swiftmag = 21
dmag = 0.05
bbmag1 = 5*u.ABmag
bbmag_target = 22*u.ABmag
while bbmag1 < bbmag_target:
    swiftmag+=dmag
    bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)

band1_fluence, foo = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)
req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('{} seconds'.format(req_exp.value))

# Note, a bit faster than previous FOM. Probably due to improved bandpass range?

151.18485439999114 seconds


In [20]:
# FOM: Time to 22 ABmag for this configuration for DUET1
# Old FOM for this 
duet = config.Telescope()
duet.read_noise = 3


bandone = duet.bandpass1
bandtwo = duet.bandpass2

siglimit=10
swiftmag = 21
dmag = 0.05
bbmag1 = 5*u.ABmag
bbmag_target = 22*u.ABmag
while bbmag1 < bbmag_target:
    swiftmag+=dmag
    bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)


# Low Zodi, 10-sigma case
[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)
req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('Low-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))


# High Zodi, 10-sigma case.
siglimit=10
[bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)
req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))

# High Zodi, 5-sigma case
siglimit=5
[bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)

req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))



Low-Zodi 10-sig, 472.7113937270214 seconds
High-Zodi 10-sig, 964.1895106193286 seconds
High-Zodi 5-sig, 261.494643494549 seconds
